In [4]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
import math
from tqdm.notebook import tqdm
import sys

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена: 
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ. 
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size] 
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
a = torch.LongTensor([1,3,4,5])
a.to(device)

tensor([1, 3, 4, 5], device='cuda:0')

In [6]:
with open('name_rus.txt', encoding='cp1251') as csvfile:
    csv_reader = csv.reader(csvfile)    
    text = list(n[0].lower() for n in csv_reader)

In [5]:
text

['авдокея',
 'авдоким',
 'авдоня',
 'авдотька',
 'авдотьюшка',
 'авдотья',
 'авдоха',
 'авдоша',
 'авдуля',
 'авдуся',
 'аграфена',
 'аграфенка',
 'агрипинка',
 'агрипка',
 'агриппа',
 'агриппина',
 'агриппич',
 'агриппична',
 'ада',
 'адам',
 'адамушка',
 'адамыч',
 'адаша',
 'аделина',
 'адя',
 'аида',
 'аидка',
 'акилина',
 'акилинка',
 'аксиньюшка',
 'аксинья',
 'аксюня',
 'аксюта',
 'аксюха',
 'аксюша',
 'акся',
 'акулина',
 'акулинка',
 'акуля',
 'ала',
 'албин',
 'албина',
 'алевтин',
 'алевтина',
 'алевтинка',
 'алевтиныч',
 'алека',
 'алексана',
 'александр',
 'александра',
 'александрин',
 'александрина',
 'александринка',
 'александрушка',
 'александрыч',
 'алексаня',
 'алексаха',
 'алексаша',
 'алексеич',
 'алексей',
 'алексейка',
 'алексий',
 'алексина',
 'алексинка',
 'алексюха',
 'алексюша',
 'алекся',
 'алена',
 'аленка',
 'аленя',
 'алеха',
 'алеша',
 'алина',
 'алинка',
 'алиса',
 'алиска',
 'алла',
 'альба',
 'альбин',
 'альбина',
 'альбинка',
 'альбиныч',
 'алюня',


In [7]:
names_train, names_test = train_test_split(text, test_size=0.2, shuffle=True, random_state=123)

In [7]:
padd_idx = 1
paddds_len1 = 8
eoss_idx = 7
[padd_idx] * paddds_len1 + [eoss_idx]

[1, 1, 1, 1, 1, 1, 1, 1, 7]

In [8]:
class Vocab:
     def __init__(self, data):
        tokens = set()
        self.max_seq_len = 0
        for item in data:
            tokens.update(item)
            self.max_seq_len = max(self.max_seq_len, len(item))
        self.unk_token = '<UNK>'
        self.pad_token = '<PAD>'
        self.sos_token = '<SOS>'
        self.eos_token = '<EOS>'
        self.idx_to_token = dict(enumerate(tokens, 4))
        self.idx_to_token[1] = self.unk_token
        self.idx_to_token[0] = self.pad_token
        self.idx_to_token[2] = self.sos_token
        self.idx_to_token[3] = self.eos_token
        self.token_to_idx = {token: idx for idx, token in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)
        
        
        
class SurnamesDataset(Dataset):
    def __init__(self, X, vocab):
                     self.X = X
                     self.vocab = vocab
    def to_sequence(self, surname):
        sos_idx = self.vocab.token_to_idx[self.vocab.sos_token]
        tok_seq = [self.vocab.token_to_idx[tok] for tok in surname]
        padds_len = self.vocab.max_seq_len - len(tok_seq)
        pad_idx = self.vocab.token_to_idx[self.vocab.pad_token]
        eos_idx = self.vocab.token_to_idx[self.vocab.eos_token]
        padds = [pad_idx] * padds_len + [eos_idx]
        return torch.LongTensor(tok_seq + padds)
    def to_sequence1(self, surname):
        sos_idx = self.vocab.token_to_idx[self.vocab.sos_token]
        tok_seq1 = [self.vocab.token_to_idx[tok] for tok in surname][1:]
        padds_len1 = self.vocab.max_seq_len - len(tok_seq1)
        pad_idx = self.vocab.token_to_idx[self.vocab.pad_token]
        eos_idx = self.vocab.token_to_idx[self.vocab.eos_token]
        padds1 = [pad_idx] * padds_len1 + [eos_idx]
        return torch.LongTensor(tok_seq1 + padds1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.to_sequence(self.X[idx]), self.to_sequence1(self.X[idx])

In [9]:
# print(input_seq, dict_size, seq_len, batch_size)

In [10]:
vocab = Vocab(names_train)

In [11]:
vocab.token_to_idx['<SOS>']

2

In [12]:
train_dataset = SurnamesDataset(names_train, vocab)
test_dataset = SurnamesDataset(names_test, vocab)

In [13]:
list(train_dataset)

[(tensor([26,  8, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([ 8, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([19, 15, 31, 25, 22,  0,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([15, 31, 25, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([28, 15, 28, 14, 11, 22,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([15, 28, 14, 11, 22,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([ 9,  5, 33, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([ 5, 33, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([11, 25, 28,  5, 18,  0,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([25, 28,  5, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([18, 20, 18, 14, 27, 15,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([20, 18, 14, 27, 15,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([29, 15, 18, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3]),
  tensor([15, 18, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3])),
 (tensor([29, 11, 25, 18, 15,  0,  0,  0,

In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [15]:
list(train_loader)[0]

[tensor([[ 4, 25, 31, 19, 15, 18,  0,  0,  0,  0,  0,  0,  0,  3],
         [15,  8, 25, 11, 20, 18, 15,  0,  0,  0,  0,  0,  0,  3],
         [28, 15, 11, 25, 31, 12, 22, 18,  0,  0,  0,  0,  0,  3],
         [19,  5,  6, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
         [20, 31, 15, 20,  8,  9, 15,  0,  0,  0,  0,  0,  0,  3],
         [15, 31,  6, 15, 19,  5, 16, 15,  0,  0,  0,  0,  0,  3],
         [19, 25, 11, 15, 18, 15,  0,  0,  0,  0,  0,  0,  0,  3],
         [33, 28, 25,  6, 14, 33, 22,  0,  0,  0,  0,  0,  0,  3],
         [19, 15,  9, 33, 20, 19, 20, 15, 18,  0,  0,  0,  0,  3],
         [ 8, 20, 19, 15, 27, 15,  0,  0,  0,  0,  0,  0,  0,  3],
         [ 4, 25, 18, 18, 15,  8, 12, 20, 24,  0,  0,  0,  0,  3],
         [33, 28, 25,  6, 14, 16, 15,  0,  0,  0,  0,  0,  0,  3],
         [14, 11, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
         [20, 18, 22,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3],
         [18, 15,  6, 15, 18, 22,  0,  0,  0,  0,  0,  0,  0, 

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [66]:
class RNNClassifierCustomRNN(nn.Module):
     def __init__(self, vocab_len, embedding_size, rnn_hidden_size, output_size):
        super(RNNClassifierCustomRNN, self).__init__()
        self.rnn_hidden_size = rnn_hidden_size
        self.embedding_size = embedding_size
        self.vocab_len = vocab_len
        self.embedding = nn.Embedding(num_embeddings = vocab_len, embedding_dim = embedding_size, padding_idx=0)
        self.rnn = nn.GRU(embedding_size, rnn_hidden_size, batch_first = True)
        self.classifier = nn.Linear(in_features=rnn_hidden_size, out_features=output_size)
        self.flattener = nn.Flatten(start_dim=0, end_dim=-2)
        
        
     def forward(self, X, h=None, c=None):
        output = self.embedding(X)
        if not (h is None):
            output, (hn) = self.rnn(output, h)
        else:
            output, (hn) = self.rnn(output)
        output = self.flattener(output)
        output = self.classifier(output)
        return output, hn

In [67]:
vocab_len = vocab.vocab_len
embedding_size = 32
rnn_hidden_size = 32
output_size = vocab.vocab_len
net1 = RNNClassifierCustomRNN(vocab_len, embedding_size, rnn_hidden_size, output_size)
net1.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(net1.parameters(), lr = 0.001)

num_epochs = 20

In [68]:
def train(model, device, train_dl, optimizer, total):
    #set model in train() mode:
    model.train()
    
    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0
    
    for i, (inputs, targets) in tqdm(enumerate(train_dl), total= total, desc='Training minibatch loop '):
        # Forward pass
        targets = torch.flatten(targets.type(torch.LongTensor))
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)[0]
        loss = criterion(outputs, targets)
        
        # Backward and optimize        
        # zero grad before new step        
        optimizer.zero_grad()                        
        loss.backward()
        optimizer.step()   

        # calculating the total_loss for checking
        total_loss += loss           
        
        # PREDICTIONS 
        total_samples += targets.shape[0]   
        _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
        correct_samples += torch.sum(predictions_indices==targets)

    train_accuracy = float(correct_samples) / total_samples        
    
    return total_loss, train_accuracy      

In [69]:
# TESTING THE MODEL
def test(model, device, test_dl, total):
    #set model in eval() mode (it skips Dropout etc):
    model.eval()
    
    total_samples = 0.0
    correct_samples = 0.0 
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(test_dl), total=total, desc='Testing minibatch loop:'):                      
            # apply model to input data
            targets = torch.flatten(targets.type(torch.LongTensor))
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)[0]        
                       
            #PREDICTIONS
            total_samples += targets.shape[0]   
            _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
            correct_samples += torch.sum(predictions_indices==targets)                    
        
    test_accuracy = correct_samples / total_samples        
    
    return test_accuracy 

In [70]:
def predict(model, max_len):
    
    chars = []
    for i in range(5):
        h= None
        with torch.inference_mode():
            total_sequence = torch.tensor([[vocab.token_to_idx['<SOS>']]], dtype=torch.int64).to(device)
            for _ in range(5):
                logits, h= net1(total_sequence, h)
        prob = nn.functional.softmax(logits[-1], dim=0).data
        # Taking the class with the highest probability score from the output
        char_ind = torch.multinomial(prob, num_samples=1)
        if vocab.idx_to_token[int(char_ind[0])] not in ['<SOS>', '<EOS>', '<PAD>', '<UNK>']:
            chars.append(vocab.idx_to_token[int(char_ind[0])])
    return chars

In [71]:
predict(net1, 5)

['у', 'п', 'г', 'а', 'ж']

In [73]:
%%time
for_losses = []
n_total_steps = len(train_loader)


for epoch in range(100):
    total_loss, train_accuracy = train(net1, device, train_loader, optimizer, total = math.ceil(len(train_dataset)/64))
    test_accuracy = test(net1, device, test_loader, total = math.ceil(len(test_dataset)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    if (epoch + 1) % 5 == 0:
        for _ in range(5):
            print(predict(net1, 12))
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/25 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3
